In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from functools import partial

In [ ]:
def generate_synthetic_mri_data(batch_size, image_shape=(1, 64, 64, 64)):
    """
    Generates pairs of synthetic 3D MRI data and regression targets.
    
    Args:
        batch_size (int): The number of data pairs to generate.
        image_shape (tuple): The shape of the MRI images.
    
    Returns:
        Tuple[Tensor, Tensor, Tensor]: A tuple containing the first set of MRI images,
        the second set of MRI images, and the regression targets.
    """
    x1 = torch.randn(batch_size, *image_shape)
    x2 = torch.randn(batch_size, *image_shape)
    y1 = torch.randn(batch_size, 1)  
    y2 = torch.randn(batch_size, 1)  

    return x1, x2, y1, y2


In [ ]:
# Model preparation - 3D resnet 101
def conv3x3x3(in_planes, out_planes, stride=1):
    """3x3x3 convolution with padding"""
    return nn.Conv3d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

def conv1x1x1(in_planes, out_planes, stride=1):
    """1x1x1 convolution"""
    return nn.Conv3d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, downsample=None, dropout_rate=0.0):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride
        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, downsample=None, dropout_rate=0.0):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1x1(in_planes, planes)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = conv3x3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = conv1x1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm3d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.dropout = nn.Dropout3d(p=dropout_rate)

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000, dropout_rate=0.0, in_channels=3):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv3d(in_channels, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1, dropout_rate=dropout_rate)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dropout_rate=dropout_rate)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dropout_rate=dropout_rate)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dropout_rate=dropout_rate)
        self.avgpool = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm3d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dropout_rate=0.0):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm3d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, dropout_rate=dropout_rate))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, dropout_rate=dropout_rate))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

def resnet101(**kwargs):
    """Constructs a ResNet-101 model."""
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    return model


In [ ]:
class ModelComb(nn.Module):
    """
    A combined model that takes two inputs, processes them through the same ResNet model,
    and outputs the features from a layer before the last two layers of the ResNet model,
    along with the final regression outputs.
    
    Args:
        resnet (nn.Module): A ResNet model used for processing the inputs.
        n_classes (int): Number of classes or output features from the ResNet model.
        n_input_channels (int): Number of input channels expected by the ResNet model.
    """
    
    def __init__(self, resnet, n_classes=400, n_input_channels=1):
        super(ModelComb, self).__init__()
        self.resnet = resnet
        
    def forward(self, x1, x2):
        # Process each input through the ResNet model
        reg0 = self.resnet(x1)
        reg1 = self.resnet(x2)
        
        # Remove the last two layers from the ResNet model for feature extraction
        feature_extractor = nn.Sequential(*list(self.resnet.children())[:-2])
        features_x1 = feature_extractor(x1)  # Extracted features for x1
        features_x2 = feature_extractor(x2)  # Extracted features for x2
        
        return features_x1, features_x2, reg0, reg1

In [ ]:
model = ModelComb(resnet=resnet101(num_classes=1, in_channels=1), n_classes=1, n_input_channels=1)

In [ ]:
def contrastive_loss(features_x1, features_x2, margin=1.0):
    """
    A simple implementation of contrastive loss.
    
    Args:
        features_x1 (Tensor): Extracted features from the first set of images.
        features_x2 (Tensor): Extracted features from the second set of images.
        margin (float): Margin for contrastive loss.
    
    Returns:
        Tensor: The computed contrastive loss.
    """
    distances = F.pairwise_distance(features_x1, features_x2, p=2)
    return torch.mean(torch.clamp(margin - distances, min=0.0))

mse_loss_fn = torch.nn.MSELoss()


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
batch_size = 8
x1, x2, y1, y2 = generate_synthetic_mri_data(batch_size)

model.train()
optimizer.zero_grad()

# Forward pass
features_x1, features_x2, reg0, reg1 = model(x1, x2)

# Compute losses
contr_loss = contrastive_loss(features_x1, features_x2)
mse_loss = mse_loss_fn(reg0, y1)
total_loss = contr_loss + mse_loss

# Backward pass and optimize
total_loss.backward()
optimizer.step()

print(f'Total Loss: {total_loss.item()}')
